In [1]:
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [2]:
if is_notebook():
    import ipyturtle3 as turtle 
else:
    import turtle 

import math
import sys

In [3]:
# Config
N_POINTS = 7
RADIUS = 200

In [4]:
# Rainbow Generator (from rainbow.ipynb)
import pandas as pd

basecolours_data = [
    (0.0, 255, 0,   0),   #red    
    (1.0, 255, 165, 0),   #orange 
    (2.0, 255, 255, 0),   #yellow 
    (3.0, 0,   128, 0),   #green  
    (4.0, 0,   0,   255), #blue   
    (5.0, 75,  0,   130), #indigo 
    (6.0, 238, 130, 238)  #violet 
]

def generateRainbowColours(n_colours):
    basecolours = pd.DataFrame(basecolours_data, columns = ['position', 'R', 'G', 'B'])
    
    # multilpe the positions of the base colours to spread them evently across the 
    # full range of target colours.
    basecolours['position'] = basecolours['position'] * (n_colours-1)/6

    # add a tag column to show these are base colours
    basecolours['base'] = True

    # create the full length data frame to hold output colours, positions 0 to n_colours exclusive
    outputcolours = pd.DataFrame({'position':range(0,n_colours)}) 

    # merge the base colours and the, currently empty, output colours. Then index on the position
    allcolours = pd.concat([basecolours, outputcolours]).set_index('position')

    # The magic is here!
    # Use interpolate to fill in the missing colours. Use the index to indicate the "spacing"
    # between colours.
    allcolours = allcolours.interpolate(method='index')

    # remove the base colours to get the final output colours
    return allcolours[allcolours.base != True].drop('base', axis=1).itertuples(index=False)

def generateRainbowColoursList(n_colours):
    return list(generateRainbowColours(n_colours))

In [5]:
# Mystic Rose Drawing Functions
pi = math.pi

def distance(i, j, n_points):
    return min( abs(i-j), n_points - abs(i-j))

def PointsInCircle(r, n=10, start=0):
    for i in range(start, n):
        yield {
            "i": i,
            "coords": (math.cos(2*pi/n*i)*r, math.sin(2*pi/n*i)*r) 
        }
        
def decimalColour(t):
    return tuple(ti/255 for ti in t)
        
def uniqueLines(radius, n_points):
    lines = []
    for pointA in PointsInCircle(radius, n_points):
        for pointB in PointsInCircle(radius, n_points, pointA["i"]+1):
            lines.append({
                "dist": distance(pointA["i"], pointB["i"], n_points),
                "from": pointA,
                "to": pointB
            })
    return lines

def drawMysticRose(radius, n_points, turtle):
    
    lines = sorted(uniqueLines(radius, n_points), key=lambda d: - d['dist'])

    max_dist = int(lines[0]['dist'])
    colours = list(map(decimalColour, generateRainbowColours(max_dist)))
    
    for line in lines:
        if int(line["dist"]) <= len(colours):
            turtle.pencolor(colours[line["dist"]-1])
        else:
            turtle.pencolor("black")
            
        turtle.penup()
        #display(line["from"]["coords"])
        turtle.goto(line["from"]["coords"])
        turtle.pendown()
        turtle.goto(line["to"]["coords"])
        turtle.penup()
    return len(lines) #same as int(n_points * (n_points-1) / 2)

In [6]:
def getNewTurtleAndDisplay():
    myTurtle = None
    
    if is_notebook():
        myCanvas = turtle.Canvas(width=RADIUS*2,height=RADIUS*2)
        display(myCanvas)
        myTurtleScreen = turtle.TurtleScreen(myCanvas)
        myTurtleScreen.delay(1)
        myTurtle = turtle.Turtle(myTurtleScreen)

    else:
        wn = turtle.Screen()
        wn.tracer(False)

    myTurtle.speed(2)
    return myTurtle

In [7]:
myTurtle = getNewTurtleAndDisplay()
lines = drawMysticRose(RADIUS, 6, myTurtle)
f"{lines} lines"

Canvas(height=400, width=400)

'15 lines'

In [8]:
myTurtle = getNewTurtleAndDisplay()
lines = drawMysticRose(RADIUS, 7, myTurtle)
f"{lines} lines"

Canvas(height=400, width=400)

'21 lines'

In [9]:
myTurtle = getNewTurtleAndDisplay()
lines = drawMysticRose(RADIUS, 8, myTurtle)
f"{lines} lines"

Canvas(height=400, width=400)

'28 lines'

In [10]:
myTurtle = getNewTurtleAndDisplay()
lines = drawMysticRose(RADIUS, 9, myTurtle)
f"{lines} lines"

Canvas(height=400, width=400)

'36 lines'

In [11]:
myTurtle = getNewTurtleAndDisplay()
lines = drawMysticRose(RADIUS, 10, myTurtle)
f"{lines} lines"

Canvas(height=400, width=400)

'45 lines'

In [13]:
myTurtle = getNewTurtleAndDisplay()
lines = drawMysticRose(RADIUS, 30, myTurtle)
f"{lines} lines"

Canvas(height=400, width=400)

'435 lines'

In [12]:
# Avoid screen closing
if not is_notebook():    
    turtle.getscreen()._root.mainloop()  # <-- run the Tkinter main loop

NOTE
https://github.com/altair-viz/altair/issues/329#issuecomment-473524751